# **Beauty продукты**

Предложите 3 способа найти упоминания товаров в отзывах. Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара. Могут помочь заголовки и дополнительные данные с Amazon (Metadata здесь) Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?

Реализуйте один из предложенных вами способов.

Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика ранжирует выше коллокации, подходящие для отчёта).

Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров. Должны получиться примерно такие группы:

watch 
--- 
stylish watch

good watches

great watch

love this watch
... *Курсив*

**Способы найти упоминания товаров в отзывах:**


1. Правиловый подход на основе шаблонов типа "мыло Х", "духи Y" и подобных, получится что-то вроде словаря основных слов для поиска. Словарь можно расширить при помощи эмбеддингов - найти ближайшие синонимы. Для этого понадобится только почистить тексты и "пройтись" по ним этими шаблонами. Потом надо будет из шаблонов выделить само название/NE. Достоинства: довольно просто и быстро, как раз тексты одной тематики и предметная область ограничена. Недостаток: в отзывах очень редко встречаются названия товаров, все пишут только "этот продукт", "это" и т.п + далеко не всегда после слов "мыло", "духи" и т.п. будет идти название товара.
2. Выделить кандидатов в NE (например, именные группы), а потом их классифицировать, опираясь, например, на грамматические признаки, символьные признаки или контекст. Достоинства: с таким количеством доп. признаков и хорошей моделью классификации, скорее всего, должно быть высокое качество. Недостатки: такой подход может сильно зависеть от тематики и формы текстов, выделять тщательно кандидатов долго и трудно, а также подбирать подходящую модель классификации.
3. Разметка последовательности: каждое слово как бы воспринимается как кандидат в NE, то есть размечаются все входные токены (как NE или не NE, или с использованием BIO-разметки: начало, середина, конец NE). Достоинства: на выходе получаем больше информации, чем в других методах. Недостатки: сложность модели, мне кажется. 
4. Можно использовать специальные инструменты для распознавания сущностей (например, natasha\yargy), которые, скорее всего, достаточно точно выделят названия товаров. Достоинства: на среднестатистических текстах наташа работает достаточно хорошо, ее можно менять в процессе и подстраивать под задачу. Недостатки: в отзывах опять же очень мало упоминаний конкретных названий + возможно, понадобятся свои правила.

Для всех этих подходов необходимы просто тексты и для некоторых дополнительно полученные значения признаков типа POS-тэга или данных о заглавной букве в начале слова.

А ещё везде можно смотреть на заголовки! Там всегда есть название товара.





Я хочу попробовать взять слова из заголовков (названий товаров) и к ним уже применить расширение, добавив ближайших слов с помощью эмбеддингов. Данные использую не все, оставила 10000 отзывов (всю выборку перемешала и обрезала).

Читалка данных, преобразуем в датафрейм с нужными нам полями

In [ ]:
with open('Beauty_crop.txt', 'r', encoding='utf-8') as f:
    all_data = f.read()
    reviews = all_data.split('\n\n')

In [ ]:
import pandas as pd
import tqdm

In [ ]:
df = pd.DataFrame(columns=['id', 'title', 'text'])
for review in tqdm.tqdm(reviews):
    review = review.split('\n')
    df.loc[len(df)] = [review[0].split(':', 1)[1].strip(), review[1].split(':', 1)[1].strip(), review[-1].split(':', 1)[1].strip()]

100%|██████████| 10000/10000 [00:33<00:00, 295.34it/s]


К сожалению, данные ещё пришлось обрезать, потому что дальнейшие функции работают неприлично долго.

In [ ]:
import random

In [ ]:
df = df.sample(3000)

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

,id,title,text
0,B0009OAHIW,Dolce & Gabbana Light Blue,I bought this because it was considerably chea...
1,B000GHWX34,D & G Light Blue By Dolce & Gabbana For Women....,"Light Blue is a glorious scent, based in apple..."
2,B0001TSIR2,Dr. Bronner'S Organic Bar Soaps Pure Castile,If you want to use a natural soap that leaves ...
3,B000EB8CLE,Neutrogena Rainbath Refreshing Shower and Bath...,When I ordered this product I was expecting th...
4,B0007URYFM,Boundaries,This book is published by Zondervan; there are...


Чистим заголовки и тексты от пунктуации и стоп-слов, лемматизируем

In [ ]:
import nltk 
from nltk.corpus import stopwords

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from string import punctuation

In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
english_stopwords = stopwords.words("english")
english_stopwords = english_stopwords + ['fl', 'oz', 'ml', 'ounce', 'ounces', 'pounds', 's']

In [ ]:
def clean(text):
    lemmas = []
    reg = re.compile('[^a-zA-Z ]')
    doc = nlp(text)
    for tok in doc:
        if tok.lower_ not in punctuation and tok.lower_ not in english_stopwords:
            if tok.lemma_ == "-PRON-":
                tok = tok.lower_
            else:
                tok = reg.sub('', tok.lemma_.lower().strip())
            tok = tok.strip()
            if tok != '':
                lemmas.append(tok)
    return ' '.join(lemmas)

In [ ]:
df.title[0], clean(df.title[0])

('Dolce & Gabbana Light Blue', 'dolce gabbana light blue')

In [ ]:
tqdm.tqdm.pandas()

In [ ]:
df['clean_title'] = df['title'].progress_apply(clean)
df['clean_text'] = df['text'].progress_apply(clean)

100%|██████████| 3000/3000 [01:01<00:00, 48.60it/s]


In [ ]:
df.head()

,id,title,text,clean_title,clean_text
0,B0009OAHIW,Dolce & Gabbana Light Blue,I bought this because it was considerably chea...,dolce gabbana light blue,buy considerably cheap usually purchase also f...
1,B000GHWX34,D & G Light Blue By Dolce & Gabbana For Women....,"Light Blue is a glorious scent, based in apple...",g light blue dolce gabbana women deodorant spray,light blue glorious scent base apple musk jasm...
2,B0001TSIR2,Dr. Bronner'S Organic Bar Soaps Pure Castile,If you want to use a natural soap that leaves ...,dr bronner s organic bar soaps pure castile,want use natural soap leave clean without chem...
3,B000EB8CLE,Neutrogena Rainbath Refreshing Shower and Bath...,When I ordered this product I was expecting th...,neutrogena rainbath refreshing shower bath gel,order product expect regular version nothing t...
4,B0007URYFM,Boundaries,This book is published by Zondervan; there are...,boundary,book publish zondervan biblical reference cros...


Теперь для каждого слова из (очищенного) названия попробуем добавить ближайшее слово в модели ворд2век от гугла. Из названия, потому что там наверняка встретится самая важная NE для товара.

In [ ]:
import gensim

In [ ]:
!gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
1.65GB [00:08, 195MB/s]


In [ ]:
! gunzip GoogleNews-vectors-negative300.bin.gz

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, unicode_errors='replace', limit=200000)

In [ ]:
def add_similar(title):
    thesaurus = []
    for token in title.split():
        thesaurus.append(token)
        if token in model:
            thesaurus.append(model.most_similar(token)[0][0].lower())
    return thesaurus

In [ ]:
df['thesaurus'] = df['clean_title'].progress_apply(add_similar)

100%|██████████| 3000/3000 [06:19<00:00,  7.90it/s]


In [ ]:
df.head()

,id,title,text,clean_title,clean_text,thesaurus
0,B0009OAHIW,Dolce & Gabbana Light Blue,I bought this because it was considerably chea...,dolce gabbana light blue,buy considerably cheap usually purchase also f...,"[dolce, gabbana, light, lights, blue, red]"
1,B000GHWX34,D & G Light Blue By Dolce & Gabbana For Women....,"Light Blue is a glorious scent, based in apple...",g light blue dolce gabbana women deodorant spray,light blue glorious scent base apple musk jasm...,"[g, h, light, lights, blue, red, dolce, gabban..."
2,B0001TSIR2,Dr. Bronner'S Organic Bar Soaps Pure Castile,If you want to use a natural soap that leaves ...,dr bronner s organic bar soaps pure castile,want use natural soap leave clean without chem...,"[dr, mr, bronner, s, €_™_s, organic, organic, ..."
3,B000EB8CLE,Neutrogena Rainbath Refreshing Shower and Bath...,When I ordered this product I was expecting th...,neutrogena rainbath refreshing shower bath gel,order product expect regular version nothing t...,"[neutrogena, rainbath, refreshing, invigoratin..."
4,B0007URYFM,Boundaries,This book is published by Zondervan; there are...,boundary,book publish zondervan biblical reference cros...,"[boundary, boundaries]"


Теперь будем выделять нграммы (ближайшие соседи слева и справа, так как они показались мне наиболее значимыми: например, "лучшее мыло" или "мыло прекрасно")

In [ ]:
def ngrams(text, thesaurus):
    ngrams = {}
    text = text.split()
    for i, token in enumerate(text):
        if token in thesaurus:
            ngrams[token] = []
            if i != 0:
                ngrams[token].append(text[i-1] + ' ' + token)
            if i+1 != len(text):
                ngrams[token].append(token + ' ' + text[i+1])
    return ngrams

In [ ]:
df['ngrams'] = df[['clean_text', 'thesaurus']].progress_apply(lambda row: ngrams(row['clean_text'], row['thesaurus']), axis=1)





100%|██████████| 3000/3000 [00:00<00:00, 32297.75it/s]


In [ ]:
df.head()

,id,title,text,clean_title,clean_text,thesaurus,ngrams
0,B0009OAHIW,Dolce & Gabbana Light Blue,I bought this because it was considerably chea...,dolce gabbana light blue,buy considerably cheap usually purchase also f...,"[dolce, gabbana, light, lights, blue, red]",{}
1,B000GHWX34,D & G Light Blue By Dolce & Gabbana For Women....,"Light Blue is a glorious scent, based in apple...",g light blue dolce gabbana women deodorant spray,light blue glorious scent base apple musk jasm...,"[g, h, light, lights, blue, red, dolce, gabban...","{'light': ['spray light', 'light enough'], 'bl..."
2,B0001TSIR2,Dr. Bronner'S Organic Bar Soaps Pure Castile,If you want to use a natural soap that leaves ...,dr bronner s organic bar soaps pure castile,want use natural soap leave clean without chem...,"[dr, mr, bronner, s, €_™_s, organic, organic, ...","{'soap': ['brand soap', 'soap miss']}"
3,B000EB8CLE,Neutrogena Rainbath Refreshing Shower and Bath...,When I ordered this product I was expecting th...,neutrogena rainbath refreshing shower bath gel,order product expect regular version nothing t...,"[neutrogena, rainbath, refreshing, invigoratin...",{}
4,B0007URYFM,Boundaries,This book is published by Zondervan; there are...,boundary,book publish zondervan biblical reference cros...,"[boundary, boundaries]",{'boundary': ['talk boundary']}


Решила дальше все смотреть и демонстрировать на примере топ-10 самых представленных в выборке продуктов

In [ ]:
most_represented = []
groups = df.groupby('id')
for k, v in sorted(groups.groups.items(), key=(lambda kv: len(kv[1])), reverse=True)[:10]:
    most_represented.append(k)
    print(k, v)

B0009V1YR8 Int64Index([  53,   54,  272,  399,  441,  550,  563,  596,  621,  652,  674,
             700, 1106, 1396, 1452, 1472, 1507, 1687, 1688, 1691, 1738, 1777,
            1893, 2162, 2195, 2291, 2376, 2438, 2562, 2602, 2715, 2780, 2793,
            2852, 2898, 2982, 2985, 2995],
           dtype='int64')
B0000YUXI0 Int64Index([ 181,  257,  348,  898, 1238, 1371, 1382, 1420, 1690, 1811, 2016,
            2160, 2540, 2559, 2644, 2773],
           dtype='int64')
B00011JL6W Int64Index([ 500,  508,  541,  715,  725,  777, 1399, 1441, 2097, 2254, 2426,
            2495, 2592, 2671, 2977],
           dtype='int64')
B0000533G8 Int64Index([325, 559, 661, 1277, 1477, 1566, 1603, 1785, 2199, 2427, 2657,
            2875],
           dtype='int64')
B00011JKS6 Int64Index([336, 538, 847, 1320, 1361, 1379, 1408, 1765, 1854, 2642], dtype='int64')
B00011JM66 Int64Index([5, 281, 289, 448, 1039, 1322, 1509, 1538, 1834, 1877], dtype='int64')
B00021DVCQ Int64Index([228, 571, 655, 766, 948, 1107, 13

Дальше посмотрим на коллокации. Так как самих биграмм было найдено очень мало для запуска такого ранжирования, я решила посмотреть на эти метрики с точки зрения ранжирования коллокаций из всех очищенных текстов.

In [ ]:
import nltk
from nltk.collocations import *

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [ ]:
finder = BigramCollocationFinder.from_documents(df['clean_text'].apply(lambda text: text.split()))

In [ ]:
finder.score_ngrams(bigram_measures.pmi)[:20]

[(('aaaa', 'aaa'), 16.726444361120034),
 (('ademas', 'rinde'), 16.726444361120034),
 (('advance', 'trish'), 16.726444361120034),
 (('aesthetic', 'atrocious'), 16.726444361120034),
 (('aire', 'libre'), 16.726444361120034),
 (('airplane', 'fuel'), 16.726444361120034),
 (('allof', 'friz'), 16.726444361120034),
 (('amanda', 'richards'), 16.726444361120034),
 (('ambree', 'legrain'), 16.726444361120034),
 (('ammonia', 'resorcinol'), 16.726444361120034),
 (('amusement', 'park'), 16.726444361120034),
 (('animalprotect', 'foeswho'), 16.726444361120034),
 (('antioxidants', 'vitamins'), 16.726444361120034),
 (('ao', 'propoacutesito'), 16.726444361120034),
 (('aproveacutechenla', 'pasas'), 16.726444361120034),
 (('aquellas', 'persona'), 16.726444361120034),
 (('arbonne', 'dhc'), 16.726444361120034),
 (('argentina', 'uruguay'), 16.726444361120034),
 (('articulo', 'inmediatamentele'), 16.726444361120034),
 (('ashe', 'cinder'), 16.726444361120034)]

In [ ]:
finder.score_ngrams(bigram_measures.student_t)[:20]

[(('can', 'not'), 12.375255885976804),
 (('have', 'use'), 10.84743091462316),
 (('use', 'product'), 10.070163717813214),
 (('work', 'well'), 10.04846144581766),
 (('highly', 'recommend'), 9.755933301085491),
 (('nail', 'polish'), 9.099427219518239),
 (('year', 'ago'), 9.029581982216827),
 (('would', 'recommend'), 8.671041103723036),
 (('will', 'not'), 8.537868360276534),
 (('smell', 'like'), 8.432134489553093),
 (('not', 'know'), 8.4097002906281),
 (('first', 'time'), 8.115156167151362),
 (('long', 'time'), 8.092364450648814),
 (('dry', 'skin'), 8.038332624547627),
 (('great', 'product'), 7.992714361722389),
 (('recommend', 'product'), 7.969880548681493),
 (('work', 'great'), 7.810082531413461),
 (('feel', 'like'), 7.693948554130004),
 (('hair', 'dryer'), 7.587115771252509),
 (('really', 'like'), 7.544499029331948)]

In [ ]:
finder.score_ngrams(bigram_measures.likelihood_ratio)[:20]

[(('can', 'not'), 1308.8268085489924),
 (('highly', 'recommend'), 995.3886084417711),
 (('nail', 'polish'), 725.771367148112),
 (('year', 'ago'), 696.5863644655271),
 (('china', 'glaze'), 568.2872731537924),
 (('flat', 'iron'), 560.9997062062223),
 (('work', 'well'), 520.5664165248829),
 (('have', 'use'), 506.977082820252),
 (('sensitive', 'skin'), 415.06004824745446),
 (('would', 'recommend'), 405.2537357914656),
 (('will', 'not'), 395.72501480321444),
 (('top', 'coat'), 382.9624267769837),
 (('curl', 'iron'), 380.4078893852437),
 (('every', 'day'), 377.7509954714794),
 (('first', 'time'), 376.6704244904686),
 (('long', 'time'), 363.492655885527),
 (('year', 'old'), 352.25179642388497),
 (('eau', 'de'), 347.4735625934911),
 (('not', 'know'), 338.05231154865),
 (('little', 'bit'), 332.4896308604349)]

Мне кажется, что метрика с критерием Стьюдента и метрика с функцией правдоподобия оказались очень похожи по результатам -- они выделяют те коллокации, которые фактически являются названиями товаров (возможно, в биграммах они просто не так широко представлены, если смотреть на них в рамках одного продукта): например, nail polish и top coat.

Вывожу все найденные биграммы для каждого слова из словаря для каждго из наиболее представленных продуктов (один продукт повторяется, так как почему-то у него разные айди)

In [ ]:
for key, value in groups.groups.items():
    if key in most_represented:
        bigrams_for_product = {}
        for index in value:
            bigrams = df['ngrams'][index]
            for word, ngram_list in bigrams.items():
                if word in bigrams_for_product:
                    bigrams_for_product[word] += ngram_list
                else:
                    bigrams_for_product[word] = []
        print(df[df.id == key].iloc[0]['title'], '\n')
        for key, value in bigrams_for_product.items():
            if len(value) > 0:
                print(key, '\n____________\n', '\n'.join(value[:5]), '\n')

Dr. Bronner - Castile Soap 

soap 
____________
 body soap
bronners soap
soap need
liquid soap
soap please 

dr 
____________
 please dr
dr bronner
use dr
dr bronner
ingredient dr 

bronner 
____________
 dr bronner
bronner s
remember bronner
bronner s
dr bronner 

castile 
____________
 castile shampoo 

Conair CHV14JXR Extreme Heat Jumbo and Super Jumbo Rollers 

roller 
____________
 top roller
roller heat
hair roller
roller burn
keep roller 

heat 
____________
 roller heat
heat fast
use heat
heat fast
time heat 

conair 
____________
 think conair
conair would
call conair
conair send
purchase conair 

jumbo 
____________
 one jumbo
jumbo one 

super 
____________
 anything super
super long 

Mavala Stop - Helps Cure Nail Biting and Thumb Sucking, 0.3-Fluid Ounce 

stop 
____________
 mavala stop
stop anyone
wife stop
stop bite
son stop 

thumb 
____________
 suck thumb
thumb anymore
dip thumb
thumb milk
suck thumb 

mavala 
____________
 apply mavala
mavala cry
else mavala
mavala 

К сожалению, эти коллокации получились не такими красивыми, как хотелось бы. Скорее всего, так получилось из-за того, что в отзывах действительно пишут в 95% случаев "этот продукт" или "это" вместо NE. 